### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0
!pip install --force-reinstall -v openai==1.12.0
!pip install httpx==0.27.2 --force-reinstall


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Using pip 24.2 from /opt/app-root/lib64/python3.11/site-packages/pip (python 3.11)
  Obtaining dependency information for openai==1.12.0 from https://files.pythonhosted.org/packages/26/a1/75474477af2a1dae3a25f80b72bbaf20e8296191ece7fff2f67984206f33/openai-1.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/a1/ee/48ca1a7c89ffec8b6a0c5d02b89c305671d5ffd8d3c94acf8b8c408575bb/anyio-4.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-non

In [3]:
import os
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import VLLMOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Milvus

#### Bases parameters, Inference server and Milvus info

In [ ]:
# Replace values according to your Milvus deployment (including the proper Milvus Collection name)
INFERENCE_SERVER_URL = "http://vllm.vllm.svc.cluster.local:8000/v1"
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
MAX_TOKENS=1024
TOP_P=0.95
TEMPERATURE=0.01
PRESENCE_PENALTY=1.03
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "pdf_collection"
API_KEY=""

#### Initialize the connection

In [19]:
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



<All keys matched successfully>


#### Initialize query chain

In [20]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant named HatBot answering questions.
You will be given a question you need to answer, and a context to provide you with information. You must answer the question based as much as possible on this context.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Context: 
{context}

Question: {question} [/INST]
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = VLLMOpenAI(
    openai_api_key=API_KEY,
    openai_api_base=INFERENCE_SERVER_URL,
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    presence_penalty=PRESENCE_PENALTY,
    streaming=True,
    verbose=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
            ),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
        )

os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### Query example

In [21]:
question = "Do i need new DNS?"
result = qa_chain.invoke({"query": question})

Based on the context provided, it appears that you are in the process of onboarding your application to Red Hat OpenShift Service Mesh (OSSM) and need to make some changes to your DNS records and git repo. The context mentions that you will need to remove the A records created for your application in Infoblox and cleanup the git repo of the egress IP manifest for your application. However, it does not explicitly state that you need new DNS.

Therefore, based on the information given, I cannot definitively answer whether you need new DNS or not. If you have any specific concerns about your DNS setup, I would recommend checking the Service Mesh App Cluster Info to confirm your cluster and namespace assignments, and then contacting the Enterprise Pipeline Support team for further assistance.

#### Retrieve source

In [22]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

docs/Service Mesh Onboarding - Platform Docs.pdf
docs/sample.md


In [16]:
question = "who can create a new ingressgateway repo and how long would it take?"
result = qa_chain.invoke({"query": question})

Based on the context provided, creating a new Ingress Gateways app repository can be done by the Application Tenant team. The context mentions that they need to add the Ingress Gateways app to their Application Tenant repo and provides an example of how to create a Git repository with a specific naming convention.

As for the time it takes to create a new repository, it depends on various factors such as network speed, server response time, and the complexity of the repository setup. Generally speaking, creating a new empty Git repository should not take long, but the time may vary. If you need a more accurate estimate, I would recommend checking with the Application Tenant team or consulting the documentation for the specific Git hosting service you are using.

In [23]:
question = "Openshift Users and Groups"
result = qa_chain.invoke({"query": question})


Based on the context provided, OpenShift uses Projects to manage access and resources for users and groups. In an OpenShift environment, you can create projects for different teams or applications, and assign specific roles to users within those projects. This helps ensure that users have the appropriate access levels for their tasks.

OpenShift supports several built-in roles, such as "cluster-admin," "build," "edit," and "view." You can also create custom roles with specific permissions. Users can be added to a project as members, and their roles can be managed through the OpenShift web console or the command-line interface.

Additionally, OpenShift supports RBAC (Role-Based Access Control), which allows you to define roles and permissions based on specific attributes, such as user identity or resource labels. This helps ensure that access is granted based on the principle of least privilege, improving security and compliance.

It's important to note that managing users and groups i

In [24]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

docs/sample.md
